# BAN5753 EXERCISE 2

## Part I

1.	Build a regression model to predict Satisf using only dealers’ perceived importance about choosing a supplier of hydraulic and pneumatic products. Variable names in the data are Reliab2, Time2, ….,Return2, Warranty2 as independent variables.

In [21]:
# import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
# import necessary package to handle excel workbook
import openpyxl
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.feature_selection import RFE, RFECV


In [17]:
# import data
df = pd.read_excel("C:\\Users\\cwolt\\OneDrive\\Desktop\\BAnDS\\Fall 2025\\Advanced Business Analytics\\Module 2\\BFP1_Data.xlsx")
df.head()

,id,colrcode,reliab,time,av_br,av_spec,price,credit,av_pay,return,...,fn_newpr,fn_need,fn_order,fn_pric,fn_other,pr_area,sales,num_emp,industry,size
0,1,8.0,8.0,8.0,6.0,7.0,7.0,5.0,8.0,7.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5.0,6.0,34.0,large
1,5,8.0,9.0,9.0,9.0,9.0,9.0,7.0,6.0,8.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,5.0,30.0,NaN
2,9,5.0,8.0,7.0,4.0,8.0,5.0,1.0,1.0,5.0,...,0.0,1.0,0.0,0.0,0.0,2.0,NaN,3.0,34.0,NaN
3,10,8.0,9.0,9.0,6.0,7.0,9.0,9.0,4.0,9.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,4.0,99.0,NaN
4,11,8.0,9.0,8.0,7.0,8.0,4.0,3.0,2.0,4.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,35.0,large


In [15]:
# Show column headers
df.columns

Index(['id', 'colrcode', 'reliab', 'time', 'av_br', 'av_spec', 'price',
       'credit', 'av_pay', 'return', 'warranty', 'talk_dir', 'reliab2',
       'time2', 'av_br2', 'av_spec2', 'price2', 'credit2', 'return2',
       'warrant2', 'satisf', 'rate', 'percent', 'i_acces', 'purch_i',
       'purch_i2', 'purchgrp', 'often', 'rout_int', 'rout_cal', 'rout_sal',
       'rout_no', 'fn_tech', 'fn_newpr', 'fn_need', 'fn_order', 'fn_pric',
       'fn_other', 'pr_area', 'sales', 'num_emp', 'industry', 'size'],
      dtype='object')

In [16]:
# transpose the outpute of the df
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,317.0,637.192429,403.014979,1.0,283.0,622.0,959.0,1355.0
colrcode,316.0,4.889241,2.597388,1.0,2.0,5.0,8.0,8.0
reliab,313.0,8.456869,1.046465,1.0,8.0,9.0,9.0,9.0
time,313.0,8.434505,0.931822,2.0,8.0,9.0,9.0,9.0
av_br,311.0,6.617363,1.750084,1.0,6.0,7.0,8.0,9.0
av_spec,311.0,7.636656,1.538686,2.0,7.0,8.0,9.0,9.0
price,313.0,7.421725,1.627422,1.0,7.0,8.0,9.0,9.0
credit,308.0,5.827922,2.248329,1.0,5.0,6.0,7.0,9.0
av_pay,309.0,3.417476,2.340458,1.0,1.0,3.0,5.0,9.0
return,310.0,6.709677,1.809603,1.0,5.0,7.0,8.0,9.0


In [18]:
model_df = df[['reliab2', 'time2', 'av_br2', 'av_spec2', 'price2', 'credit2', 'return2', 'warrant2']]
# check for null values in subsetted data
model_df.isnull().sum()

reliab2     11
time2       11
av_br2      13
av_spec2    15
price2      13
credit2     23
return2     20
warrant2    18
dtype: int64

In [19]:
# drop all null values
model_df = model_df.dropna(axis=0, how='any')

In [20]:
# checking the data types of the columns to ensure they can be used for model building
model_df.dtypes

reliab2     float64
time2       float64
av_br2      float64
av_spec2    float64
price2      float64
credit2     float64
return2     float64
warrant2    float64
dtype: object

In [22]:
# define the model formula
ols_formula = """
satisf ~ 
reliab2 + 
time2 + 
av_br2 + 
av_spec2 + 
price2 + 
credit2 + 
return2 + 
warrant2
"""
# fit the regression model and print the model diagnostics
linear_model = smf.ols(ols_formula, df).fit()
linear_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 satisf   R-squared:                       0.621
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     56.15
Date:                Wed, 27 Aug 2025   Prob (F-statistic):           1.94e-53
Time:                        21:48:53   Log-Likelihood:                -469.59
No. Observations:                 283   AIC:                             957.2
Df Residuals:                     274   BIC:                             990.0
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0266      0.481      4.213      0.000       1.080       2.974
reliab2        0.8019      0.086      9.360      0.000       0.633       0.971
time2          0.1481      0.073      2.033      0.043       0.005       0.291
av_br2        -0.0839      0.073     -1.150      0.251      -0.227       0.060
av_spec2       0.0649      0.065      1.002      0.317      -0.063       0.192
price2         0.0597      0.061      0.977      0.330      -0.061       0.180
credit2       -0.2610      0.067     -3.910      0.000      -0.392      -0.130
return2        0.2223      0.091      2.432      0.016       0.042       0.402
warrant2       0.0077      0.089      0.087      0.931      -0.167       0.183
==============================================================================
Omnibus:                       48.358   Durbin-Watson:                   2.163
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              244.073
Skew:                          -0.546   Prob(JB):                     1.00e-53
Kurtosis:                       7.417   Cond. No.                         122.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""